# IMPORTAÇÃO DE BIBLIOTECAS

In [ ]:
import pandas as pd
import re
import nltk
import spacy
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


nltk.download('punkt')
nltk.download('stopwords')
spacy.cli.download('pt_core_news_lg')
nlp = spacy.load("pt_core_news_lg")
stopwords = nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# PRÉ-PROCESSAMENTO DE TEXTO

## Limpeza de texto (Tirando espaços e pontuação)

Essa parte tem como propósito realizar a limpeza do texto de entrada, removendo caracteres não alfanuméricos e convertendo-o para minúsculas. Esse procedimento é essencial para a padronização e simplificação dos dados textuais, facilitando análises posteriores e a extração de características.

In [ ]:
def clean_text(text):
    cleaned_text = re.sub(r"[^\w\s]", "", text).lower()
    return cleaned_text

## Tokenização

Realizamos a tokenização, que é dividir o texto em palavras individuais (tokens). Essa etapa é fundamental em processamento de linguagem natural (PLN) para permitir que o modelo compreenda e processe o texto em nível de palavra, facilitando a extração de informações relevantes.

In [ ]:
def tokenizacao(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

## Remoção de "stopwords"

Aqui estamos removendo as "stopwords" porque queremos remover palavras comuns, conhecidas como stopwords, da lista de tokens, focando em palavras mais significativas. Essa prática é útil para reduzir o ruído nos dados, uma vez que stopwords geralmente não carregam um significado semântico importante.

In [ ]:
def remove_stopwords(array_tokens):
    tokens_filtered = [word for word in array_tokens if word.lower() not in stopwords]
    return tokens_filtered

## Lemmatização

 Nessa parte realizamos a lematização na lista de tokens, reduzindo as palavras à sua forma base. Esse processo é valioso para padronizar as palavras, reduzindo-as à sua forma mais básica. Isso auxilia na captura do significado central das palavras e consolida as características nos dados.

In [ ]:
def lemmatization(array_of_tokens):
    text = ' '.join(array_of_tokens)
    doc = nlp(text)
    tokens_lemma = [token.lemma_ for token in doc]
    return tokens_lemma

## Vetorização do Bag Of Words

Aqui convertemos um DataFrame de mensagens de texto processadas em uma representação de "saco de palavras" (BoW, do inglês "bag-of-words") utilizando o CountVectorizer. Essa transformação é crucial para representar dados textuais numericamente, onde cada coluna corresponde a uma palavra única, e cada linha representa um documento.

In [ ]:
def BowVectorizer(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['processed_message_str'])
    df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    df_bow['intent'] = df['intent']
    return df_bow

# Pipeline

A função 'pipeline()' combina as etapas individuais de processamento (limpeza, tokenização, remoção de stopwords, lematização) em um pipeline para facilitar a aplicação em novos textos. Essa abordagem simplifica o fluxo de trabalho de processamento de texto, melhorando a legibilidade do código e permitindo um processamento consistente em diferentes entradas.

In [ ]:
def pipeline(frase):

  p1 = clean_text(frase)
  p2 = tokenizacao(p1)
  p3 = remove_stopwords(p2)
  p4 = lemmatization(p3)
  return p4

## Criação do dataframe e aplicação do pipeline

In [ ]:
data = {
    'intent': ['show_trending_topics', 'show_trending_topics', 'show_trending_topics', 'show_trending_topics', 'show_trending_topics', 'show_trending_topics',
               'popular_videos', 'popular_videos', 'popular_videos', 'popular_videos', 'popular_videos', 'popular_videos',
               'graph_views_minutes', 'graph_views_minutes', 'graph_views_minutes', 'graph_views_minutes', 'graph_views_minutes', 'graph_views_minutes'],
    'message': [
        'quais são os assuntos do momento?',
        'mostre os tópicos mais falados agora',
        'o que está bombando nas redes sociais?',
        'me mostre os assuntos mais comentados',
        'quero ver os trending topics',
        'mostre os tópicos mais populares',

        'quais são os vídeos mais vistos?',
        'mostre os vídeos populares',
        'quais são os vídeos que estão em alta?',
        'me mostre os vídeos mais assistidos',
        'mostre os vídeos que todo mundo está assistindo',
        'quero ver os vídeos mais populares',

        'exiba o gráfico de visualizações por minuto',
        'mostre o gráfico de visualizações ao longo do tempo',
        'quero ver as visualizações por minuto',
        'mostre a evolução das visualizações',
        'exiba o gráfico que mostra as visualizações por minuto',
        'mostre as estatísticas de visualizações por minuto',
    ]
}

df = pd.DataFrame(data)

# Aplica a função 'pipeline' em cada mensagem da coluna 'message' do DataFrame 'df'
df['processed_message'] = df['message'].apply(pipeline)

# Converte a lista de tokens resultante em cada linha para uma string única separada por espaços
df['processed_message_str'] = df['processed_message'].apply(lambda x: ' '.join(x))

df_bow = BowVectorizer(df)

In [ ]:
display(df_bow)

,agora,alto,assister,assistir,assunto,bombar,comentar,estatística,evolução,exiba,...,social,tempo,todo,topics,trending,tópico,ver,visualização,vídeo,intent
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,show_trending_topics
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,show_trending_topics
2,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,show_trending_topics
3,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,show_trending_topics
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,0,0,show_trending_topics
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,show_trending_topics
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,popular_videos
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,popular_videos
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,popular_videos
9,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,popular_videos


# Função de uso do algoritmo (Usando Similaridade de Cosseno)

A função find_most_similar_intent foi desenvolvida para identificar a intenção mais semelhante a partir de uma frase de entrada, usando uma representação Bag-of-Words (BoW) do texto processado. Na primeira parte, a frase de entrada é processada através da aplicação da função pipeline, resultando em uma lista de tokens. Posteriormente, esses tokens são agrupados em uma única string, essencial para a representação BoW.

Em seguida, o vetorizador utilizado no DataFrame df_bow é reutilizado, configurado para empregar o mesmo vocabulário, excluindo a coluna 'intent'. A frase processada da entrada é transformada em um vetor BoW, preservando a consistência nas representações.

A similaridade do cosseno é então calculada entre o vetor da entrada e todos os vetores BoW das frases no DataFrame. Este procedimento gera uma matriz de similaridades, em que cada elemento reflete a similaridade entre a entrada e uma frase específica no DataFrame.

Para determinar a frase mais similar, é obtido o índice do maior valor na matriz de similaridades através do método argmax. Com base nesse índice, a função extrai a intenção correspondente ao DataFrame df_bow. Por fim, a função retorna a intenção mais provável, representando a intenção associada à frase de entrada mais semelhante no contexto das intenções previamente definidas no DataFrame. Essa abordagem é valiosa em aplicações onde se deseja classificar automaticamente a intenção de uma entrada de texto, como em sistemas de processamento de linguagem natural (PLN) e chatbots.

In [ ]:
def find_most_similar_intent(input_text, df_bow):
    # Processa a frase de entrada usando o pipeline
    input_processed = pipeline(input_text)
    input_processed_str = ' '.join(input_processed)

    # Usa o mesmo vetorizador utilizado no DataFrame
    vectorizer = CountVectorizer(vocabulary=df_bow.columns[:-1])  # Excluindo a coluna 'intent'
    input_vectorized = vectorizer.transform([input_processed_str])

    # Calcula a similaridade do cosseno entre a entrada e todas as frases no DataFrame
    similarities = cosine_similarity(input_vectorized, df_bow.iloc[:, :-1])

    # Obtém o índice da frase mais similar
    most_similar_index = similarities.argmax()

    # Obtém o intent correspondente à frase mais similar
    most_similar_intent = df_bow.iloc[most_similar_index]['intent']

    return most_similar_intent

## Input do texto

In [ ]:
# Define a mensagem de entrada que se deseja analisar
input_text = 'mostre os assuntos em alta'

# Obtém a intenção mais provável associada à mensagem de entrada
predicted_intent = find_most_similar_intent(input_text, df_bow)

# Exibe na console a mensagem indicando a intenção prevista para a mensagem de entrada
print(f'A frase "{input_text}" é mais similar ao intent: {predicted_intent}')


A frase "mostre os assuntos em alta" é mais similar ao intent: show_trending_topics
